In [1]:
import pandas as pd
import requests
from haversine import haversine
from tqdm import tqdm

In [2]:
load_lst = pd.read_excel("주변식당목록.xlsx")
load_lst

,가맹점명,주소,영업시간,분류,점수,건수
0,라메흐꺄도,서울 영등포구 선유로17길 24 신일아르디세 1층 105~106호,영업시간 매일 11:00 ~ 21:20 · 휴게시간 매일 15:30 ~ 16:30,"해물,생선",4.1,43.0
1,타이산,서울 영등포구 선유로13길 25 에이스하이테크시티2 지식산업센터 116호,NaN,중국요리,3.8,9.0
2,화사랑화로구이 문래점,"서울 영등포구 선유로9가길 16 광양림빌딩 1,2층",영업시간 매일 11:30 ~ 22:00,"육류,고기",3.5,27.0
3,화덕으로간고등어반상,서울 영등포구 선유로9길 10 문래SKV1센터 1층 119호,영업시간 월~토 11:00 ~ 21:00 · 휴게시간 월~토 15:00 ~ 17:00,"해물,생선",4.5,2.0
4,쁘띠문래,서울 영등포구 선유로17길 24 신일아르디세 1층 109호,영업시간 화~토 12:00 ~ 22:00 · 휴게시간 화~토 15:00 ~ 17:00,양식,4.1,23.0
...,...,...,...,...,...,...
414,더문,서울 영등포구 선유로 27,NaN,술집,0.0,0.0
415,그린스푼,서울 영등포구 선유로3길 10 하우스디비즈 지하1층 B101호,NaN,한식,0.0,0.0
416,먹태랑노가리,서울 영등포구 도림로 461,NaN,"호프,요리주점",0.0,0.0
417,골든푸드코리아,서울 영등포구 도림로 456 상가 101동 105호,NaN,한식,0.0,0.0


In [6]:
apiurl = "https://api.vworld.kr/req/address?"
params = {
	"service": "address",
	"request": "getcoord",
	"crs": "epsg:4326",
	"address": "서울 영등포구 선유로9길 30",
	"format": "json",
	"type": "road",
	"key": "79B583D6-112C-3703-8CB0-415188DD02E0"
}

response = requests.get(apiurl, params=params)

# 내 위치 위,경도 추출
st_x = response.json()['response']['result']['point']['x']
st_y = response.json()['response']['result']['point']['y']
st = (float(st_y),float(st_x))

# 위,경도 추출 실패한 주소 확인용 리스트
add_lst_b = []

for add in tqdm(load_lst['주소']):
    
    # api에서 받아들일 수 있게 주소 재정의
    add_ = add.split()[0]+' '+add.split()[1]+' '+add.split()[2]+' '+add.split()[3]
    params['address'] = add_
    response = requests.get(apiurl, params=params)

    try:
        en_x = response.json()['response']['result']['point']['x']
        en_y = response.json()['response']['result']['point']['y']
    except:
        add_lst_b.append(add)
        continue

    en = (float(en_y),float(en_x))
    dis = round(haversine(st,en,unit='m'),2) # 내 위치와 식당 거리 계산
    load_lst.loc[load_lst['주소']==add,'거리'] = dis

100%|██████████| 419/419 [02:40<00:00,  2.61it/s]


In [7]:
add_lst_b

['서울 영등포구 문래동6가 36 e편한세상문래 에듀플라츠 상가 1층 107호']

In [8]:
load_lst

,가맹점명,주소,영업시간,분류,점수,건수,거리
0,라메흐꺄도,서울 영등포구 선유로17길 24 신일아르디세 1층 105~106호,영업시간 매일 11:00 ~ 21:20 · 휴게시간 매일 15:30 ~ 16:30,"해물,생선",4.1,43.0,342.84
1,타이산,서울 영등포구 선유로13길 25 에이스하이테크시티2 지식산업센터 116호,NaN,중국요리,3.8,9.0,136.22
2,화사랑화로구이 문래점,"서울 영등포구 선유로9가길 16 광양림빌딩 1,2층",영업시간 매일 11:30 ~ 22:00,"육류,고기",3.5,27.0,300.36
3,화덕으로간고등어반상,서울 영등포구 선유로9길 10 문래SKV1센터 1층 119호,영업시간 월~토 11:00 ~ 21:00 · 휴게시간 월~토 15:00 ~ 17:00,"해물,생선",4.5,2.0,188.95
4,쁘띠문래,서울 영등포구 선유로17길 24 신일아르디세 1층 109호,영업시간 화~토 12:00 ~ 22:00 · 휴게시간 화~토 15:00 ~ 17:00,양식,4.1,23.0,342.84
...,...,...,...,...,...,...,...
414,더문,서울 영등포구 선유로 27,NaN,술집,0.0,0.0,338.31
415,그린스푼,서울 영등포구 선유로3길 10 하우스디비즈 지하1층 B101호,NaN,한식,0.0,0.0,265.02
416,먹태랑노가리,서울 영등포구 도림로 461,NaN,"호프,요리주점",0.0,0.0,711.60
417,골든푸드코리아,서울 영등포구 도림로 456 상가 101동 105호,NaN,한식,0.0,0.0,777.66


In [9]:
load_lst.to_excel("거리추가.xlsx", index=False)